# IBM Data Science Capstone - Toronto Neighbourhood Clustering Part 2

First starting with the importing of the libraries and variables that wlil be used when we're ready to import and clean the data.

In [58]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import json # library to handle JSON files

import requests # library to handle requests
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
#from sklearn.cluster import KMeans

print ('Imported Libraries.')

Imported Libraries.


In [2]:
# Main vars

# URL where we can find the neighbourhoods
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

geo_url = 'https://cocl.us/Geospatial_data'

# Data Frame to hold the neighbourhood list
tdot_neighbourhoods_df = []
tdot_geocodes_df = []


## Merge-a-Plenty

Time to get to work now that our libs and vars have been dealt with.  

I needed to split this off into a function to keep things relatively clean. If there's a more efficient, vector-driven way of doing this, please leave a comment I'd be happy to explore further!

In [3]:
# Data Cleansing Functions

# postcode_merge: take repeated postal codes and combine the neighbourhoods
def postcode_merge(df):
    
    # Vars needed for processing
    return_df, multi_postcode_df, duplicate_postcodes_list = [], [], []
    
    # Create list of postal codes we need to merge
    duplicate_postcodes_df = df.groupby(['Postcode','Borough']).count() > 1
    duplicate_postcodes_df = duplicate_postcodes_df[duplicate_postcodes_df['Neighbourhood'] == True]
    duplicate_postcodes_df.reset_index(inplace=True)
    duplicate_postcodes_list = duplicate_postcodes_df['Postcode'].to_numpy();
    
    # Split dataframes into done pile and merge pile
    return_df = df[~df['Postcode'].isin(duplicate_postcodes_list)][['Postcode','Borough','Neighbourhood']]
    multi_postcode_df = df[df['Postcode'].isin(duplicate_postcodes_list)][['Postcode','Borough','Neighbourhood']]
     
    # Iterate through the duplicate postcode list, NOT the dataframes themselves
    for postcode in duplicate_postcodes_list:

        # Don't repeat if we already processed the postcal code
        if return_df[return_df['Postcode'] == postcode].shape[0] == 0:
            
            # Join the neighbourhoods w/ matching postal code, sort list by neighbourhood name
            append_postcode = postcode
            append_hood =  ', '.join(sorted(map(str, multi_postcode_df[multi_postcode_df['Postcode'] == postcode]['Neighbourhood'].to_numpy())))
            append_borough = multi_postcode_df[multi_postcode_df['Postcode'] == postcode]['Borough'].unique()[0]
            
            # Add entry to return df
            return_df = return_df.append(pd.DataFrame({'Postcode':append_postcode,'Borough':append_borough,'Neighbourhood':append_hood}, index=[0]), ignore_index=True)
    
    # fix and return the clean list of postalcodes and neighbourhoods
    return return_df.rename(columns={"Postcode": "PostalCode","Neighbourhood":"Neighborhood"});
        

print ("functions compiled.")

functions compiled.


## Read and clean the data

It's now time to scrape the web page and clean up the data. I used Pandas to scrape but I would like to retry this one day soon using Beautiful Soup (Note to self: Google it).


In [4]:
# Read the Wikipedia data
tdot_neighbourhoods_df = pd.read_html(url)[0]

# Rule 1: Drop unassigned boroughs
tdot_neighbourhoods_df.drop( tdot_neighbourhoods_df[ tdot_neighbourhoods_df['Borough'] == 'Not assigned' ].index , inplace=True)

# Rule 2: Merge the repeated postal codes
tdot_neighbourhoods_df = postcode_merge(tdot_neighbourhoods_df)

# Rule 3: Fill unassigned hoods
tdot_neighbourhoods_df['Neighborhood'][tdot_neighbourhoods_df['Neighborhood'] == 'Not assigned'] = tdot_neighbourhoods_df['Borough'];

# Just some final sorting touches
tdot_neighbourhoods_df.sort_values(by=['Borough','Neighborhood'], inplace=True)
tdot_neighbourhoods_df.reset_index(drop=True, inplace=True)
print ("")


## Toronto Neighbourhoods by Postal Code

The end result with all cleansing in the rearview mirror...

In [5]:
tdot_neighbourhoods_df

,PostalCode,Borough,Neighborhood
0,M4S,Central Toronto,Davisville
1,M4P,Central Toronto,Davisville North
2,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi..."
3,M5P,Central Toronto,"Forest Hill North, Forest Hill West"
4,M4N,Central Toronto,Lawrence Park
5,M4T,Central Toronto,"Moore Park, Summerhill East"
6,M5R,Central Toronto,"North Midtown, The Annex, Yorkville"
7,M4R,Central Toronto,North Toronto West
8,M5N,Central Toronto,Roselawn
9,M5H,Downtown Toronto,"Adelaide, King, Richmond"


### Lets dbl-check the row count

We'll check the row count using .shape

In [6]:
print ("Number of rows: ", tdot_neighbourhoods_df.shape[0])

Number of rows:  103


## Geocoding the Toronto Postal Codes
This code was supplied by IBM and modified / scaled up.  The geocodeer library was etremely unreliable - replaced with CSV.


In [7]:
# import geocoder
#!pip install --upgrade geocoder

#import geocoder 


#def get_coords(postal_code):

    # initialize your variable to None
#    lat_lng_coords = None

    # loop until you get the coordinates
 #   while(lat_lng_coords is None):
  #      print('{}, Toronto, Ontario'.format(postal_code))
   #    print (g.latlng)
    #    lat_lng_coords = g.latlng
    
    #return [lat_lng_coords[0], lat_lng_coords[1]];

#tdot_neighbourhoods_df['Latitude','Longitude'][0] = get_coords(tdot_neighbourhoods_df['PostalCode'][0])

#print ("Done.")
#tdot_neighbourhoods_df

In [8]:
tdot_geocodes_df = pd.read_csv(geo_url)

In [9]:
# Merge the dataframes by using the postal codes

tdot_geocodes_df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
tdot_neighbourhoods_df = pd.merge(tdot_neighbourhoods_df, tdot_geocodes_df, on='PostalCode')
tdot_neighbourhoods_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4S,Central Toronto,Davisville,43.704324,-79.388790
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049
3,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
6,M5R,Central Toronto,"North Midtown, The Annex, Yorkville",43.672710,-79.405678
7,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
8,M5N,Central Toronto,Roselawn,43.711695,-79.416936
9,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568


# Mapping Some Results
Now let's have some fun with Maps. We start by adding some Folium into the mix.

In [10]:
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Additional libraries imported.')
print('Folium installed')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [14]:
# The code was removed by Watson Studio for sharing.

Foursquare credentials loaded.


#### Use geopy library to get the latitude and longitude values of Toronto

In [15]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [23]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
neighborhoods = tdot_neighbourhoods_df;

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Focus on Downtown Toronto
We're getting somewhere, but now let's check out the features of neighbourhoods in downtown Toronto

In [24]:
downtown_data = neighborhoods[neighborhoods['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
1,M5V,Downtown Toronto,"Bathurst Quay, CN Tower, Harbourfront West, Is...",43.628947,-79.394420
2,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
3,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [26]:
downtown_data.shape

(19, 5)

In [27]:
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [32]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, label in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

### Venues for each neighbourhood
This function is compliments of the good folks at IBM to access the Foursquare API that will provide us with venue information about the Downtown Toronto Borough.

In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Now cycle through the Neighbourhoods, grabbing venue infomation

In [37]:
# type your answer here

toronto_venues = getNearbyVenues(names=downtown_data['Neighborhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )


Adelaide, King, Richmond
Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara
Berczy Park
Cabbagetown, St. James Town
Central Bay Street
Chinatown, Grange Park, Kensington Market
Christie
Church and Wellesley
Commerce Court, Victoria Hotel
Design Exchange, Toronto Dominion Centre
First Canadian Place, Underground city
Garden District, Ryerson
Harbord, University of Toronto
Harbourfront
Harbourfront East, Toronto Islands, Union Station
Queen's Park
Rosedale
St. James Town
Stn A PO Boxes 25 The Esplanade


In [65]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",30,30,30,30,30,30
"Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara",17,17,17,17,17,17
Berczy Park,30,30,30,30,30,30
"Cabbagetown, St. James Town",30,30,30,30,30,30
Central Bay Street,30,30,30,30,30,30
"Chinatown, Grange Park, Kensington Market",30,30,30,30,30,30
Christie,18,18,18,18,18,18
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30


In [39]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 150 uniques categories.


### Analyze each neighbourhood

In [41]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Liquor Store,Lounge,Market,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skating Rink,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Adelaide, King, Richmond",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Adelaide, King, Richmond",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Adelaide, King, Richmond",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Adelaide, King, Richmond",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Adelaide, King, Richmond",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


This function was provided by the good folks at IBM, for returning the top n venue types per neighbourhood

In [44]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Liquor Store,Lounge,Market,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skating Rink,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.100000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
1,"Bathurst Quay, CN Tower, Harbourfront West, Is...",0.000000,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)

    return row_categories_sorted.index.values[0:num_top_venues]

### What are the most popular venue types?

In [54]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Steakhouse,Hotel,Asian Restaurant,Seafood Restaurant,Noodle House,Opera House,Lounge,Pizza Place,Café,Concert Hall
1,"Bathurst Quay, CN Tower, Harbourfront West, Is...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Sculpture Garden,Rental Car Location,Bar,Plane,Boutique
2,Berczy Park,Seafood Restaurant,Bakery,Coffee Shop,Cocktail Bar,Beer Bar,Farmers Market,Park,Concert Hall,Cheese Shop,Café
3,"Cabbagetown, St. James Town",Restaurant,Café,Italian Restaurant,Coffee Shop,Park,Market,Pub,Butcher,Playground,Caribbean Restaurant
4,Central Bay Street,Coffee Shop,Chinese Restaurant,Bubble Tea Shop,Japanese Restaurant,Italian Restaurant,Spa,Gastropub,Bar,Park,Modern European Restaurant


### Let's cluster the neighbourhoods to see what we can conclude

In [55]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = downtown_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,1,Steakhouse,Hotel,Asian Restaurant,Seafood Restaurant,Noodle House,Opera House,Lounge,Pizza Place,Café,Concert Hall
1,M5V,Downtown Toronto,"Bathurst Quay, CN Tower, Harbourfront West, Is...",43.628947,-79.394420,0,Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Sculpture Garden,Rental Car Location,Bar,Plane,Boutique
2,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Seafood Restaurant,Bakery,Coffee Shop,Cocktail Bar,Beer Bar,Farmers Market,Park,Concert Hall,Cheese Shop,Café
3,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,1,Restaurant,Café,Italian Restaurant,Coffee Shop,Park,Market,Pub,Butcher,Playground,Caribbean Restaurant
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,3,Coffee Shop,Chinese Restaurant,Bubble Tea Shop,Japanese Restaurant,Italian Restaurant,Spa,Gastropub,Bar,Park,Modern European Restaurant
5,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049,1,Café,Vietnamese Restaurant,Bakery,Mexican Restaurant,Comfort Food Restaurant,Caribbean Restaurant,Cheese Shop,Cocktail Bar,Coffee Shop,Organic Grocery
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564,1,Grocery Store,Café,Park,Athletics & Sports,Nightclub,Candy Store,Restaurant,Diner,Italian Restaurant,Baby Store
7,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,Gay Bar,Burger Joint,Creperie,Breakfast Spot,Restaurant,Bubble Tea Shop,Ramen Restaurant,Pizza Place,Gastropub,Coffee Shop
8,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,1,Café,Coffee Shop,Restaurant,Gastropub,Deli / Bodega,Japanese Restaurant,Gym / Fitness Center,Steakhouse,Bakery,Pub
9,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,1,Coffee Shop,Café,Restaurant,Deli / Bodega,Japanese Restaurant,Gym / Fitness Center,Pub,Salad Place,Beer Bar,Hotel Bar


### Let's visualize these clusters

In [66]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=20,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Top Insights

<ul>
    <li>If you higly value outdoor recreation and wish to be as close to the downtown core as possible, you want to live in <b>Rosedale</b>.  It is by and large the most outdoor recreation-friendly neighbourhood in the downtown core, 
        with enough outdoor venue types to set it apart as its own cluster.
    </li>
    <li>There may be some meaningful insights to uncover re: the abundance of coffee shops in the financial district.
    </li>
</ul>
    

### I hope you enjoyed it!